## Ride-hailing Simulation

In [23]:
from collections import OrderedDict
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from lifelines import KaplanMeierFitter
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

ride_hailing_dataset_path = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\Ride_hailing_Dataset.csv'
ACTIVE = 'active'
WORKING = 'working'
INACTIVE = 'inactive'
NO_TRIP_ID = -1

### Time dependent Cox Model

In [25]:
from collections import OrderedDict
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ride_hailing_dataset_path = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\Ride_hailing_Dataset.csv'
ACTIVE = 'active'
WORKING = 'working'
INACTIVE = 'inactive'
NO_TRIP_ID = -1
import numpy as np
from lifelines import CoxTimeVaryingFitter

filtered_data=pd.read_csv(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\Shifted_Data.csv')
#filtered_data=filtered_data.drop(columns=['trip_request','distance_y','cumulative_fare'])

#filtered_data['total_trips'] = filtered_data.groupby('ID')['trip_request'].cumsum()
filtered_data=filtered_data.drop(columns=['trip_request','distance_y','fare'])

# Initialize the CoxTimeVaryingFitter
ctv = CoxTimeVaryingFitter()

# Fit the model
try:
    ctv.fit(filtered_data, id_col='ID', event_col='event', start_col='start', stop_col='stop', show_progress=True)

except Exception as e:
    print(f"Error fitting the Cox model: {e}")


def calculate_survival_probability(duration, shifted_fare, shifted_trip_request, ctv):
    """
    Calculate the survival probability for a driver given the duration on the platform, 
    fare earned, and trip requests using a fitted Cox time-dependent model.

    Parameters:
    duration (float): The duration (in hours) the driver has been on the platform.
    shifted_fare (float): The fare amount earned by the driver.
    shifted_trip_request (int): The number of trip requests completed by the driver.
    ctv (CoxTimeVaryingFitter): The fitted Cox time-dependent model.

    Returns:
    float: The survival probability for the given parameters.
    """
    # Coefficients from the Cox model
    coef_fare = 0.01
    coef_trip_request = -0.18

    # Calculate the linear predictor
    eta = (coef_fare * shifted_fare) + (coef_trip_request * shifted_trip_request)

    # Compute the hazard ratio
    hazard_ratio = np.exp(eta)

    # Obtain the baseline survival probability at the given duration
    # Make sure duration is within the range of the baseline survival function
    S0_t = ctv.baseline_survival_.iloc[duration].values[0]

    # Calculate the survival probability
    survival_probability = S0_t ** hazard_ratio

    return survival_probability*0.7

# Example usage:
# Assuming `ctv` is the fitted CoxTimeVaryingFitter model
#duration = 120
#shifted_fare = 0
#shifted_trip_request = 0
#prob = calculate_survival_probability(duration, shifted_fare, shifted_trip_request, ctv)
#print(f"Survival Probability: {prob}")


c:\Users\xshuo\anaconda3\lib\site-packages\lifelines\utils\__init__.py:1033: RuntimeWarning: There exist rows in your DataFrame with start and stop both at time 0:

        >>> df.loc[(df[start_col] == df[stop_col]) & (df[start_col] == 0)]

        These can be safely dropped, which should improve performance.

        >>> df = df.loc[~((df[start_col] == df[stop_col]) & (df[start_col] == 0))]
  warnings.warn(warning_text, RuntimeWarning)


Iteration 1: norm_delta = 1.07e-01, step_size = 0.9500, log_lik = -640088.38930, newton_decrement = 1.19e+02, seconds_since_start = 1.8
Iteration 2: norm_delta = 2.37e-03, step_size = 0.9500, log_lik = -639971.69479, newton_decrement = 3.76e-01, seconds_since_start = 3.9
Iteration 3: norm_delta = 1.19e-04, step_size = 0.9500, log_lik = -639971.31905, newton_decrement = 9.80e-04, seconds_since_start = 6.1
Iteration 4: norm_delta = 6.28e-06, step_size = 1.0000, log_lik = -639971.31807, newton_decrement = 2.45e-06, seconds_since_start = 8.2
Iteration 5: norm_delta = 3.69e-11, step_size = 1.0000, log_lik = -639971.31807, newton_decrement = 3.61e-17, seconds_since_start = 10.4
Convergence completed after 5 iterations.


## add interzonal time

## define class Driver, TripRequest, Simulation

In [10]:
from collections import OrderedDict
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ride_hailing_dataset_path = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\Ride_hailing_Dataset.csv'
ACTIVE = 'active'
WORKING = 'working'
INACTIVE = 'inactive'
NO_TRIP_ID = -1

## Fit driver survival model

In [6]:
class Driver:
    def __init__(self, name, current_location, start_time):
        self.name = name
        self.current_location = current_location
        # Q: Can we What is the diff between is_available = False and an 'inactive' state?
        # A: available means whether a driver is available for accepting new trip;
        # not available == inactive or working state
        # available == active state
        # hence is_available == (state == 'active')
        self.is_available = False  # Initially, all drivers are available.
        self.start_time = start_time
        self.activity_log = OrderedDict() # {timestep1: [state, id, start_location, end_location]} OrderedDict[Int: List]
        self.state = 'active'  # 'active', 'working', 'inactive'
        self.last_survival_probability = 1.0  # Initial survival probability is 1 (100%)
        self.fare = 0.0  # Fare earned by the driver
        self.trip_requests = 0

    def log_activity(self, time_step, activity):
        self.activity_log[time_step] = activity

    def update_availability(self, current_time):
        # Check if the driver's start time has passed and update availability
        if current_time >= self.start_time and self.state == 'active':
            self.is_available = True
        else:
            self.is_available = False


    def calculate_survival_probability(duration, shifted_fare, shifted_trip_request, ctv):
        # Coefficients from the Cox model
        coef_fare = 0.01
        coef_trip_request = -0.18

        # Calculate the linear predictor
        eta = (coef_fare * shifted_fare) + (coef_trip_request * shifted_trip_request)

        # Compute the hazard ratio
        hazard_ratio = np.exp(eta)

        # Obtain the baseline survival probability at the given duration
        # Make sure duration is within the range of the baseline survival function
        S0_t = ctv.baseline_survival_.iloc[duration].values[0]
        # Calculate the survival probability
        survival_probability = S0_t ** hazard_ratio

        return survival_probability*0.6


    def assign_to_trip(self, time_step, trip_request):
        self.is_available = False
        self.state = 'working'
        activity = [self.state, trip_request.ID, trip_request.start_location, trip_request.end_location]
        self.trip_requests=1
        self.fare = 0
        self.trip_requests = 0
        
        self.log_activity(time_step, activity)

    def complete_trip(self, time_step, trip_request):
        self.current_location = trip_request.end_location
        self.is_available = True
        self.state = 'active'

        
        self.fare = trip_request.fare
        self.trip_requests = 1
      


        activity = [self.state, trip_request.ID, trip_request.start_location, trip_request.end_location]
        self.log_activity(time_step, activity)
        # Update survival probability
       
     


    def leave_platform(self, time_step):
        self.is_available = False
        self.state = 'inactive'
        activity = [self.state, "NO_TRIP_ID", self.current_location, self.current_location]
        self.log_activity(time_step, activity)
         # Log the transition from active to inactive
       
        
    # def leave_platform_due_to_duration(self, current_time):
    #     WORKING_HOUR_PER_DRIVER = 20
    #     if current_time - self.start_time >= WORKING_HOUR_PER_DRIVER:
    #         self.leave_platform(current_time)


#   #  def return_to_the_platform(self, current_time):
#         self.current_location = trip_request.end_location
#         self.is_available = True
#         self.state = 'active'
#         activity = [self.state, trip_request.ID, trip_request.start_location, trip_request.end_location]
#         self.log_activity(time_step, activity)

    # def decide_to_accept_trip(self, trip_request):
    #     # Implement logic for the driver to decide whether to accept or reject the trip
    #     # For example, we can use a random decision here. You can replace this with more complex logic.
    #     decision = random.choice([True, False])
    #     return decision
    
    

    def leave_platform_due_to_duration(self,ctv, current_time):
        # Calculate total time on the platform in the correct units (hours)
        total_duration = current_time - self.start_time
        print('current_time',current_time)
        print('self.start_time',self.start_time)
        print(total_duration)
        # Calculate the new survival probability using the Cox model
        new_survival_probability = calculate_survival_probability(total_duration, self.fare, self.trip_requests, ctv)

        # Calculate the conditional probability
        if self.last_survival_probability > 0:  # Prevent division by zero
            conditional_probability = new_survival_probability / self.last_survival_probability
        else:
            conditional_probability = 1  # If last survival probability is zero, set to 1 (certain leave)

        # Update the last recorded survival probability to the new survival probability
        self.last_survival_probability = new_survival_probability

        # Generate a random number between 0 and 1
        random_chance = np.random.rand()

        # If the random chance is greater than the conditional probability, the driver decides to leave
        if random_chance > conditional_probability:
            self.leave_platform(current_time)
            print(f"{self.name} has left the platform due to survival probability analysis.")



class TripRequest:
    def __init__(self, ID, request_time, duration, start_location, end_location, distance, fare):
        self.ID = ID
        self.request_time = request_time
        self.duration = duration
        self.start_location = start_location
        self.end_location = end_location
        self.assigned_driver = None  # is an instance of Driver class
        self.is_completed = False
        self.pickup_time = None  # Will be set when assigned
        self.assigned_time = None  # Will be set when completed
        self.distance = distance  # Will be set when completed
        self.fare = distance*1.174+9.248  # Will be set when completed
        self.is_canceled = False

class Simulation:
    def __init__(self):
        # TODO: use dict {driver_id: driver_instance} to record drivers.
        self.active_drivers = OrderedDict()
        self.working_drivers = OrderedDict()
        self.inactive_drivers = OrderedDict()
        self.current_time = 0
        self.requesting_trips = []
        self.serviced_trips = []
        self.travel_time_lookup = {}
       # self.add_travel_time_lookup_from_csv()


    def add_travel_time_lookup_from_csv(self, csv_file):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file)
        
        # Iterate through the DataFrame rows
        for index, row in df.iterrows():
            # Update the travel time lookup dictionary
            # The key is a tuple of (Origin, Destination)
            # The value is the travel time 'Data'
            self.travel_time_lookup[(row['Origin'], row['Destination'])] = int(row['Data'])



    # TODO: replace add driver with driver generation function.
    # Currently, add_driver is only executed once when Simulation is instantiated. Thus, there is no need to
    # tell the driver state, as for a newly instantiated driver, its state is always active.
    # Is there any future use of this function?
    def add_driver(self, driver):
        assert driver.state == 'active'
        self.active_drivers[driver.name] = driver
        activity = [driver.state, NO_TRIP_ID, driver.current_location, driver.current_location]
        driver.log_activity(driver.start_time, activity)


  #  def add_driver(self, driver):
   #     if driver.name in self.inactive_drivers:
   #         # Reactivate the driver if they are in the inactive drivers list
   #         reactivated_driver = self.inactive_drivers.pop(driver.name)
   #         reactivated_driver.state = 'active'
   #         reactivated_driver.is_available = True
   #         self.active_drivers[reactivated_driver.name] = reactivated_driver
   #         print(f"{reactivated_driver.name} has been reactivated.")
   #     else:
    #        assert driver.state == 'active'
    #        self.active_drivers[driver.name] = driver
    #        activity = [driver.state, "NO_TRIP_ID", driver.current_location, driver.current_location]
    #        driver.log_activity(driver.start_time, activity)

            
    # TODO: remove driver as per driver survival model
    def remove_driver(self, driver_name, time_step):
        driver = self.active_drivers[driver_name]
        driver.leave_platform( time_step)
        self.active_drivers.pop(driver_name)
        self.inactive_drivers[driver_name] = driver


    # TODO: add trip request as per trip generation function
    def add_trip_request(self, trip_request):
        self.requesting_trips.append(trip_request)

    # TODO: seperate driver-trip matching policy from environment class
    def assign_driver_to_trip(self, strategy='greedy'):
        # Method to select the assignment strategy
        if strategy == 'greedy':
            self.assign_driver_by_greedy()
        elif strategy == 'availability':
            self.assign_driver_by_availability()
        else:
            print("Invalid strategy specified.")

    # TODO: seperate driver-trip matching policy from environment class
    def assign_driver_by_greedy(self):
        # assign driver by minimal travel time from driver location to passenger pickup location
        for trip_request in self.requesting_trips:
        # Cancel trip requests that have been waiting for over 15 minutes
            if self.current_time - trip_request.request_time > 15:
                trip_request.is_canceled = True
                continue

            if trip_request.request_time <= self.current_time and not trip_request.assigned_driver and not trip_request.is_canceled:
                best_driver = None
                min_travel_time = float('inf')


                for driver in list(self.active_drivers.values()):
                    # Check if the driver is available before considering them for assignment
                    if driver.is_available:
                        pair = (driver.current_location, trip_request.start_location)
                        travel_time_to_pickup = self.travel_time_lookup.get(pair)
                        travel_time_to_pickup = 0 if travel_time_to_pickup is None else travel_time_to_pickup
                       # assert travel_time_to_pickup is not None, f"There is no inter-zonal distance in lookup for driver current location and trip request location {(driver.current_location, trip_request.start_location)}"
                        if travel_time_to_pickup < min_travel_time:
                            best_driver = driver
                            min_travel_time = travel_time_to_pickup

                if best_driver:
                    # Assign the best driver to the trip
                    trip_request.assigned_time = self.current_time
                    trip_request.pickup_time = self.current_time + min_travel_time
                    trip_request.assigned_driver = best_driver

                    best_driver.assign_to_trip(trip_request.pickup_time, trip_request)
                    # Since the driver is now working on a trip, they are no longer available
                    best_driver.is_available = False

                    # Move the driver from active to working drivers
                    self.active_drivers.pop(best_driver.name)
                    self.working_drivers[best_driver.name] = best_driver

                 #   print(f"Assigned {best_driver.name} to trip request {trip_request.ID} from {trip_request.start_location} to {trip_request.end_location}.")


    # TODO: rename function: set_offline_driver_by_survival_model
    def check_active_drivers_duration(self):
        # list() is deepcopy itself, no need to have another deepcopy
        for driver in list(self.active_drivers.values()):  # Use a copy for safe iteration
            if driver.is_available:
                driver.leave_platform_due_to_duration(ctv, self.current_time)
                
            if driver.state == 'inactive':
                self.active_drivers.pop(driver.name)
                self.inactive_drivers[driver.name] = driver

    # TODO: seperate driver-trip matching policy from environment class
    def assign_driver_by_availability(self):
        for trip_request in self.requesting_trips:
            if trip_request.request_time <= self.current_time and not trip_request.assigned_driver and self.active_drivers:
                driver_to_assign = self.active_drivers.pop(0)
                travel_time_to_pickup = self.travel_time_lookup.get((driver_to_assign.current_location, trip_request.start_location), 0)
                trip_request.assigned_time=self.current_time
                trip_request.pickup_time = self.current_time + travel_time_to_pickup
                trip_request.assigned_driver = driver_to_assign
                driver_to_assign.assign_to_trip(trip_request.pickup_time, trip_request)
                self.working_drivers.append(driver_to_assign)
             #   print(f"Assigned {driver_to_assign.name} to trip request {trip_request.ID} from {trip_request.start_location} to {trip_request.end_location}.")


    def update_all_drivers_states(self):
        for trip_request in self.requesting_trips[:]:
          #  print("trip", trip_request.ID, "driver", trip_request.assigned_driver)
            if trip_request.assigned_driver and self.current_time >= (trip_request.pickup_time + trip_request.duration):
             #   print("current_time", self.current_time)
                trip_request.is_completed = True
                trip_request.assigned_driver.complete_trip(self.current_time, trip_request)
                self.serviced_trips.append(trip_request)
                self.requesting_trips.remove(trip_request)
                if self.working_drivers.get(trip_request.assigned_driver.name):
                    self.working_drivers.pop(trip_request.assigned_driver.name)
                    self.active_drivers[trip_request.assigned_driver.name] = trip_request.assigned_driver
              #  print(f"{trip_request.assigned_driver.name} completed trip {trip_request.ID} to {trip_request.end_location}.")




    def run(self, steps):
        for _ in range(steps):
            # Update each driver's availability based on the current simulation time
            #print("time", self.current_time)
            self.update_all_drivers_states()
            # TODO: is self.update_all_drivers_states() covering driver.update_availability() ?
            # Is updating driver instance reflected on driver on self.active_drivers? Yes.
            # list() is shallow copy itself
            for driver in list(self.active_drivers.values()):
                #print(driver.name, driver.is_available)
                driver.update_availability(self.current_time)

            self.check_active_drivers_duration()  # Check and potentially remove drivers first

        #    print("active_drivers before assign", self.active_drivers.keys())
        #    print("working_drivers before assign", self.working_drivers.keys())
         #   print("inactive_drivers before assign", self.inactive_drivers.keys())

            self.assign_driver_to_trip(strategy='greedy')
          #  print("active_drivers after assign", self.active_drivers.keys())

          
           # print("working_drivers after assign", self.working_drivers.keys())
          #  print("inactive_drivers after assign", self.inactive_drivers.keys())

            self.current_time += 1

    def reset(self):
        pass

    def step(self, action):
        pass




## instantiate Simulation

In [7]:
simulation_instance = Simulation()
simulation_instance.add_travel_time_lookup_from_csv(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\aivtt.csv')

## Reading the demand CSV

In [8]:
def read_trip_requests(csv_file, start_datetime):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    df.dropna(subset=['requestdatetime','ptctripid','duration','Origin','Destination'], inplace=True)

    # Convert 'start_datetime' column to datetime
    df['requestdatetime'] = pd.to_datetime(df['requestdatetime'])

    # Define the simulation start time as the first 'start_datetime' or a predefined time
    simulation_start_time = pd.to_datetime(start_datetime)

    # Calculate time steps as the difference from the simulation start time in minutes
    df['requestdatetime'] = ((df['requestdatetime'] - simulation_start_time).dt.total_seconds() / 60).astype(int)

    # Create a list to hold TripRequest objects
    trip_requests = []

    # Iterate through the DataFrame rows and create TripRequest objects
    for index, row in df.iterrows():
        trip_request = TripRequest(
            ID=row['ptctripid'],
            request_time=row['requestdatetime'],  # Use the calculated start_time_step
            duration=int(row['duration']),
            start_location=int(row['Origin']),
            end_location=int(row['Destination']),
            distance = float(row['distance_y']),  # Will be set when completed
            fare = float(row['fareamount'])  # Will be set when completed
        )
        trip_requests.append(trip_request)

    return trip_requests

# Assuming you have the Simulation class defined as before
# Initialize the simulation
simulation_start_datetime = "2020-01-02 0:00"
# Read trip requests from the CSV file
trip_requests = read_trip_requests(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\Demand.csv',simulation_start_datetime)
# Add each trip request to the simulation
for trip_request in trip_requests:
    simulation_instance.add_trip_request(trip_request)

## Reading the Driver CSV

In [ ]:
def read_drivers(csv_file, simulation_start_datetime):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Ensure no missing values in critical columns
    df.dropna(subset=['start_datetime', 'anon_vin', 'Origin'], inplace=True)

    # Convert 'start_datetime' column to datetime
    df['start_datetime'] = pd.to_datetime(df['start_datetime'])

    # Define the simulation start time as a predefined time
    simulation_start_time = pd.to_datetime(simulation_start_datetime)

    # Calculate start time in minutes from the simulation start time
    df['start_time'] = ((df['start_datetime'] - simulation_start_time).dt.total_seconds() / 60).astype(int)

    # Create a list to hold Driver objects
    drivers = []

    # Iterate through the DataFrame rows and create Driver objects
    for index, row in df.iterrows():
        driver = Driver(
            name=row['anon_vin'],
            start_time=row['start_time'],  # Use the calculated start time in minutes
            current_location=int(row['Origin'])
        )

        drivers.append(driver)

    return drivers

# Assuming you have a simulation instance created somewhere as 'simulation_instance'
simulation_start_datetime = "2020-01-02 0:00"
drivers = read_drivers(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\driver1.csv', simulation_start_datetime)

# Add each driver to the simulation
for driver in drivers:
    simulation_instance.add_driver(driver)

# Simulation

In [ ]:
simulation_instance.run(2000)

NameError: name 'simulation_instance' is not defined

## P1 and Trip Record

In [15]:
def format_time_step(time_step):
    base_time = datetime(2020, 1, 2, 0, 0)  # Starting point of the simulation
    delta = timedelta(minutes=time_step)  # Assuming each time_step represents minutes directly
    return (base_time + delta).strftime("%Y-%m-%d %H:%M:%S")

all_drivers = {**simulation_instance.active_drivers, **simulation_instance.working_drivers, **simulation_instance.inactive_drivers}

p1_periods_data = []
for driver_name, driver in all_drivers.items():
    last_active_time_step = None  # To track when the last active period started
    last_location = None  # Initialize last known location
    for time_step, activity in driver.activity_log.items():
        state = activity[0]
        if state == 'active' and last_active_time_step is None:
            # Mark the beginning of a potential idle period
            last_active_time_step = time_step
        elif last_active_time_step is not None and state != 'active':
            # Log the idle period if transitioning out of the 'active' state
            p1_periods_data.append({
                "anon_vin": driver.name,
                "start_zone": last_location,  # Using the last known location before going to 'working' state
                "end_zone": activity[2],  # Start location of the trip
                "distance": "N/A",
                "event_type": "P1 Event",
                "idle_start_datetime": format_time_step(last_active_time_step),
                "idle_end_datetime": format_time_step(time_step)
            })
            last_active_time_step = None  # Reset after logging the idle period
        elif state == 'inactive' and last_active_time_step is not None:
            # Log the event when the driver leaves the platform
            p1_periods_data.append({
                "anon_vin": driver.name,
                "start_zone": last_location,  # Using the last known location before going to 'inactive' state
                "end_zone": activity[2],  # Current location should be same as start location when inactive
                "distance": "N/A",
                "event_type": "Driver Left Platform",
                "idle_start_datetime": format_time_step(last_active_time_step),
                "idle_end_datetime": format_time_step(time_step)
            })
            last_active_time_step = None  # Reset after logging the idle period
        last_location = activity[3]  # Always update the last known location

df_p1_periods = pd.DataFrame(p1_periods_data)
df_p1_periods.head()

df_p1_periods.to_csv(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\P2.csv', index=False)


In [16]:
import pandas as pd
from datetime import datetime, timedelta

# Assuming the format_time_step function definition as before
def format_time_step(time_step):
    base_time = datetime(2020, 1, 2, 0, 0)  # Adjust according to your simulation's start date and time
    delta = timedelta(minutes=time_step)  # Ensure time_step matches the intended unit (e.g., minutes)
    return (base_time + delta).strftime("%Y-%m-%d %H:%M:%S")

# Combine requesting and serviced trips
all_trips = simulation_instance.requesting_trips + simulation_instance.serviced_trips

trip_data = []
for trip_request in all_trips:
    if trip_request.is_completed:
        if trip_request.assigned_driver:
            trip_data.append({
                "request_ID": trip_request.ID,
                "anon_vin": trip_request.assigned_driver.name,
                "start_zone": trip_request.start_location,
                "end_zone": trip_request.end_location,
                "distance": trip_request.distance,  # Consider implementing distance calculation if applicable
                "fare": trip_request.fare,  # Consider implementing distance calculation if applicable
                "event_type": "Complete trip",
                "request_datetime": format_time_step(trip_request.request_time),
                "driver_acceptance_datetime": format_time_step(trip_request.assigned_time),
                "passenger_pickup_datetime": format_time_step(trip_request.pickup_time),
                "passenger_drop_datetime": format_time_step(trip_request.pickup_time + trip_request.duration)
            })
    elif trip_request.is_canceled:
        trip_data.append({
            "request_ID": trip_request.ID,
            "anon_vin": None,  # No driver assigned
            "start_zone": trip_request.start_location,
            "end_zone": trip_request.end_location,
            "distance": None,  # No trip completed
            "fare": None,  # No trip completed
            "event_type": "Canceled trip",
            "request_datetime": format_time_step(trip_request.request_time),
            "driver_acceptance_datetime": None,
            "passenger_pickup_datetime": None,
            "passenger_drop_datetime": None
        })

# Convert to DataFrame
df_trip_data = pd.DataFrame(trip_data)

# Optionally, display the first few rows to check the data
df_trip_data.head()


df_trip_data.to_csv(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\trip2.csv', index = False)

## Criteria Output

### 1. Average Passenger Wait Time

In [17]:
import pandas as pd

def calculate_average_waiting_time(csv_file):
    # Load the data from CSV
    df = pd.read_csv(csv_file)

    # Ensure datetime columns are in the correct format
    df["request_datetime"] = pd.to_datetime(df["request_datetime"])
    df["passenger_pickup_datetime"] = pd.to_datetime(df["passenger_pickup_datetime"])

    # Calculate waiting time in minutes
    df["waiting_time"] = (df["passenger_pickup_datetime"] - df["request_datetime"]).dt.total_seconds() / 60

    # Calculate average waiting time
    average_waiting_time = df["waiting_time"].mean()

    return average_waiting_time

# Example usage
trip_data = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\trip2.csv'
average_waiting_time = calculate_average_waiting_time(trip_data)
print(f"The average waiting time for each trip is {average_waiting_time} minutes.")


The average waiting time for each trip is 2.480958730365215 minutes.


### 2. Average Driver Idle Time

In [18]:
import pandas as pd

def calculate_mean_idle_time_all_drivers(csv_file):
    # Load the data from CSV
    df = pd.read_csv(csv_file)

    # Convert datetime columns to the correct format
    df["idle_start_datetime"] = pd.to_datetime(df["idle_start_datetime"])
    df["idle_end_datetime"] = pd.to_datetime(df["idle_end_datetime"])

    # Calculate idle time in minutes for each event
    df["idle_time_minutes"] = (df["idle_end_datetime"] - df["idle_start_datetime"]).dt.total_seconds() / 60

    # Calculate mean idle time across all drivers
    mean_idle_time_all_drivers = df["idle_time_minutes"].mean()

    return mean_idle_time_all_drivers

# Example usage
csv_file_path = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\P2.csv'
mean_idle_time_all_drivers = calculate_mean_idle_time_all_drivers(csv_file_path)
print(f"Mean Idle Time for All Drivers: {mean_idle_time_all_drivers} minutes")


Mean Idle Time for All Drivers: 13.923282862596654 minutes


### 3. Utilization Rate

In [19]:
def calculate_utilization_rate(trips_csv_file, idle_csv_file):
    # Load the trips data
    trips_df = pd.read_csv(trips_csv_file)
    trips_df["passenger_pickup_datetime"] = pd.to_datetime(trips_df["passenger_pickup_datetime"])
    trips_df["passenger_drop_datetime"] = pd.to_datetime(trips_df["passenger_drop_datetime"])
    trips_df["trip_time"] = (trips_df["passenger_drop_datetime"] - trips_df["passenger_pickup_datetime"]).dt.total_seconds() / 60

    # Load the idle data
    idle_df = pd.read_csv(idle_csv_file)
    idle_df["idle_start_datetime"] = pd.to_datetime(idle_df["idle_start_datetime"])
    idle_df["idle_end_datetime"] = pd.to_datetime(idle_df["idle_end_datetime"])
    idle_df["idle_time"] = (idle_df["idle_end_datetime"] - idle_df["idle_start_datetime"]).dt.total_seconds() / 60

    # Calculate total trip time and total idle time
    total_trip_time = trips_df["trip_time"].sum()
    total_idle_time = idle_df["idle_time"].sum()

    # Calculate utilization rate
    utilization_rate = total_trip_time / (total_trip_time + total_idle_time)

    return utilization_rate

# Example usage
trips_csv_file_path = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\trip2.csv'
idle_csv_file_path = r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\P2.csv'
utilization_rate = calculate_utilization_rate(trips_csv_file_path, idle_csv_file_path)
print(f"Utilization Rate: {utilization_rate}")


Utilization Rate: 0.4955400890868597


### 4. Percentage of Cancelled Trips

In [20]:
import pandas as pd

# Load the dataset
data = pd.read_csv(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\New folder\result\trip2.csv')

# Calculate the total number of records
total_records = len(data)

# Calculate the number of records with 'Canceled trip' in the event_type column
canceled_trips = len(data[data['event_type'] == 'Canceled trip'])

# Calculate the percentage of canceled trips
percentage_canceled = (canceled_trips / total_records) * 100

print(f"Percentage of Canceled Trips: {percentage_canceled:.2f}%")


Percentage of Canceled Trips: 0.00%


### 5. Active Trip Time Ratio

In [22]:
import pandas as pd
df2=pd.read_csv(r'D:\OneDrive - University of Toronto\City Simulation\2024.02.04 City of Toronto Simulation Platform\Dataset\Ride_hailing_Dataset.csv')


# Convert to datetime
df2['passengerpickupdatetime'] = pd.to_datetime(df2['passengerpickupdatetime'])
df2['passengerdropoffdatetime'] = pd.to_datetime(df2['passengerdropoffdatetime'])
df2['start_datetime'] = pd.to_datetime(df2['start_datetime'])
df2['end_datetime'] = pd.to_datetime(df2['end_datetime'])

# Calculate the sum of the differences
sum_trip_duration = (df2['passengerdropoffdatetime'] - df2['passengerpickupdatetime']).sum()
sum_shift_duration = (df2['end_datetime'] - df2['start_datetime']).sum()

# Convert the sums to minutes
sum_trip_duration_minutes = sum_trip_duration.total_seconds() / 60
sum_shift_duration_minutes = sum_shift_duration.total_seconds() / 60

sum_trip_duration_minutes/ sum_shift_duration_minutes

0.3227428279382164